In [1]:
from datascience import *
from datascience.predicates import are
path_data = '../../../../data/'
import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from urllib.request import urlopen
import re
from bancor3_simulator.protocol import BancorV3
from bancor3_simulator.core.settings import GlobalSettings
from decimal import Decimal
from fractions import Fraction
import pandas as pd

# Recall from an earlier chapter that we already defined the cooldown period and the exit fees as follows.
cooldown_time: int = 7
withdrawal_fee: Decimal = Decimal('0.0025')
whitelisted_tokens: list = ['bnt', 'eth', 'wbtc', 'link']
bnt_funding_limit: Decimal = Decimal('40000')
bnt_min_liquidity: Decimal = Decimal('10000')
price_feeds = pd.DataFrame({'INDX':[0],
                            'bnt':[2.5],
                            'link':[15.00],
                            'eth':[2500.00],
                            'wbtc':[40000.00]})

network_fee: Decimal = Decimal('0.2')
trading_fee: Decimal = Decimal('0.01')

# There are other possible configuration settings available, however for the present purpose we will use the defaults.
v3 = BancorV3(whitelisted_tokens=whitelisted_tokens,
              network_fee=network_fee,
              trading_fee=trading_fee,
              cooldown_time=cooldown_time,
              withdrawal_fee=withdrawal_fee,
              bnt_min_liquidity=bnt_min_liquidity,
              bnt_funding_limit=bnt_funding_limit,
              price_feeds=price_feeds)

v3.create_user('Alice')
v3.create_user('Bob')
v3.create_user('Charlie')

# The actual deposits.

v3.stake(tkn_amt=100, tkn_name='eth', user_name='Alice')

v3.stake(tkn_amt=10000, tkn_name='link', user_name='Charlie')

v3.stake(tkn_amt=100, tkn_name='wbtc', user_name='Bob')

v3.stake(tkn_amt=1, tkn_name='eth', user_name='Alice')

v3.stake(tkn_amt=1, tkn_name='link', user_name='Charlie')

v3.stake(tkn_amt=1, tkn_name='wbtc', user_name='Bob')

v3.dao_msig_init_pools(whitelisted_tokens)

v3.create_user('Trader')

v3.trade(swap_amount=2000, source_token='bnt',  target_token='link', user_name='Trader', timestep=0)

v3.trade(swap_amount=302.9981, source_token='link',  target_token='bnt', user_name='Trader', timestep=0)

v3.trade(swap_amount=1, source_token='eth',  target_token='wbtc', user_name='Trader', timestep=0)

def read_url(url):
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

TypeError: tuple expected at most 1 argument, got 2

# Introduction to the Withdrawal Algorithm

After a withdrawal has been confirmed by the user, the protocol's task is to deliver the withdrawal value back to the user, either entirely in TKN, or as a combination of TKN and BNT. The design of Bancor 3 assumes that users prefer to receive their withdrawal denominated entirely in TKN. Naively, it seems impossible to return all funds denominated in TKN to the user, if the total TKN in the vault is less than the staked balance, without exacerbating the deficit further. Therefore, in cases where the vault balance of TKN is less than the amount staked, the protocol has an obligation to cover the deficit via BNT minting. While true, one could argue that the user is likely to use the BNT they are issued to perform a swap for the TKN that was withheld. In doing so, the user indeed removes additional funds from the protocol; however, such an action also opens an opportunity wherein an arbitrageur will return the pool to balance, and restore the relative deficit of the system.

The opposite is also true. Imagine the antithetical case where users provide BNT exclusively, and the protocol mints the TKN side of the pool. During a BNT deficit, a user's withdrawal may be partially reimbursed in TKN, which they may immediately swap back for BNT. As they perform the swap, the relative price of BNT is increased vs TKN, then rebalanced with external markets through the actions of arbitrageurs.

This is an important realization. The assertion that BNT must be used to compensate TKN liquidity providers directly, (and by extension, TKN to BNT providers in an imaginary version) becomes weak. Rather than give the user BNT for them to swap, the swap can be made an implicit component of the withdrawal. Further, TKN in excess can be used to recover lost BNT, and help to maintain a healthy equilibrium between the protocol and its environment.

While this reasoning is good, it is also incomplete. The value of a swap is impacted by slippage, and therefore the depth of the pool, as well as its swap fee. Moreover, the fact that a BNT minting event could be coupled with an implicit swap opens the protocol to potential abuse through withdrawal back running attacks, which artificially increases the value of a user's withdrawal. As alluded to in an earlier section, a withdrawal fee of 0.25% is introduced, and is used to evaluate the explicit exploitability of the method. In short, so long as the back running profit margin is equal to or less than the value represented by the withdrawal fee, then the protocol will allow the user to withdraw the totality of their position exclusively in TKN, regardless of the deficit. Similarly, the protocol will attempt to recover BNT by quoting it at a higher price when withdrawals are performed during a TKN surplus state, only if the arbitrage value is less than the withdrawal fee.

In addition to these considerations, the protocol also maintains an awareness of the current pool depth, and the TKN available in the vault which are not presently available as trading liquidity on the pool. The withdrawal algorithm simultaneously handles these considerations, such that the user's withdrawal is taken exclusively from non-trading vault reserves where possible, and the anticipated, rebalanced state of the pool returns to its prior depth level. One way to visualize this behavior is to imagine the continuum of both the available trading liquidity, and non-trading liquidity as a proportion of the balance on the staking ledger. Let these proportions be the x and y axes of a three-dimensional plot, where the z axis represents the relative size of any withdrawal, relative to the total balance of the staking ledger. The resulting plot describes a surface, the volume under which defines all possible withdrawals where the protocol can exercise the logic above without causing itself financial disadvantage. Importantly, the exit fee and swap fee for the specific asset undergoing withdrawal change the shape of the surface. The plot below assumes a swap fee of 0.2% and a withdrawal fee of 0.25%.

![](https://lh6.googleusercontent.com/OwDmXrZgkx0MsyPBqlpGZJLAb7ugjJB5VtBfkbr-OyoLqpO4ROFRmMNvwiQ6wD78f0NfXSDIeeR15s0rgy70yUymKYqMDqUXyZEBBVFnM4x4E_DighnSno4TU9cR1DZyk5wL_gnB)

![](https://lh3.googleusercontent.com/6k3RytOaLFnCbyjxb-3jGx5aCrLqex4gHOcFRiS8xBR83LIluS5okNChM4IgdCeogcZZsSHpV8s9aE2Btdfi_CdEx-YvvJsx1DI4GQ8YPFp7_hMfhWFS2WIDL_k29X0NNm_jIyL8)

The `group` method also allows us to classify individuals according to multiple variables. This is called *cross-classifying*.